# Это тетрадка с Кринж-парсером

Цель тетради: запарсить данные с сайта ebay по характеристикам товаров.
В резульате должны получиться 2 дадафрейма:
* Names содержит названия товаров, а также статистику продаж (цены, общее количество продаж, общая стоимость продаж, ссылки на подробные характеристики.
* Characteristics содержит данные о характеристиках товаров (Их много, не перечесть)

Импортируем библиотеки.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from random import randint
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore', message='The frame.append method is deprecated and will be removed from pandas in a future version.')

___

# Парсим первый датафрейм - Names

In [18]:
# настраиваем драйвер
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen

# Настраиваем браузер, чтобы он не грузил Java и картинки
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options, chrome_options=chrome_options)

# Делаем запрос
driver.get("https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=Smartphones&dayRange=730&endDate=1684051559220&startDate=1620979559220&categoryId=9355&offset=0&limit=50&tabName=SOLD&tz=Europe%2FMoscow")

#Атдыхаем, ждём загрузку
time.sleep(5)

C:\Users\druny\AppData\Local\Temp\ipykernel_1240\60259596.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True  # hide GUI
C:\Users\druny\AppData\Local\Temp\ipykernel_1240\60259596.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(options=options, chrome_options=chrome_options)


Тут мы ручками авторизируемся и заходим

Парсим названия и числовые характеристики (Цены, продажи)

In [15]:
df = pd.DataFrame()

In [27]:
def table_parser(soup: BeautifulSoup):
    for product in soup.find_all('tr', class_='research-table-row'):
        title = product.find('span', attrs={'data-item-id': True}).text
        price = product.find('td', class_='research-table-row__item research-table-row__avgSoldPrice').div.find_all('div')[0].text.strip()
        shipping = product.find('td', class_='research-table-row__item research-table-row__avgShippingCost').div.find_all('div')[0].text.strip()
        sales_count = product.find('td', class_='research-table-row__item research-table-row__totalSoldCount').div.find_all('div')[0].text.strip()
        sales_value = product.find('td', class_='research-table-row__item research-table-row__totalSalesValue').div.find_all('div')[0].text.strip()
        try:
            link = product.find('a', class_='research-table-row__link-row-anchor', attrs={'href': True})['href']
        except:
            link = None

        data = {"Title": title, "Price": price, "Shipping_cost": shipping, "Sales_Count": sales_count, "Sales_Value": sales_value, "Link": link}

        global df
        df = df.append(data, ignore_index=True)

In [28]:
for _ in tqdm(range(100)): #Решено спарсить 100 страниц
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table_parser(soup)
    next_button = driver.find_element(By.XPATH, '//button[@class="pagination__next"]')
    next_button.click()
    time.sleep(randint(5, 15))
    pass

 55%|█████▌    | 55/100 [10:21<08:28, 11.30s/it]


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00988893+48451]
	(No symbol) [0x0091B8A1]
	(No symbol) [0x00825058]
	(No symbol) [0x00818205]
	(No symbol) [0x0081810C]
	(No symbol) [0x00816D46]
	(No symbol) [0x008175CA]
	(No symbol) [0x0082E8A9]
	(No symbol) [0x008291EA]
	(No symbol) [0x00828FC7]
	(No symbol) [0x0087DD16]
	(No symbol) [0x0086A2BC]
	(No symbol) [0x0087C482]
	(No symbol) [0x0086A0B6]
	(No symbol) [0x00847E08]
	(No symbol) [0x00848F2D]
	GetHandleVerifier [0x00BE8E3A+2540266]
	GetHandleVerifier [0x00C28959+2801161]
	GetHandleVerifier [0x00C2295C+2776588]
	GetHandleVerifier [0x00A12280+612144]
	(No symbol) [0x00924F6C]
	(No symbol) [0x009211D8]
	(No symbol) [0x009212BB]
	(No symbol) [0x00914857]
	BaseThreadInitThunk [0x76F47D59+25]
	RtlInitializeExceptionChain [0x77A1B74B+107]

In [ ]:
df.to_csv("Names.tsv", sep='\t', index=False)

___

# Парсим характеристики моделей

In [2]:
df=pd.read_csv("Names.tsv", delimiter = "\t")

In [3]:
Links = df.Link.tolist()
Links

['https://www.ebay.com/itm/254604777645?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/254187678666?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/363183815277?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/382605209867?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/382446518910?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/401763851775?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/362534145919?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/254733458977?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/254449547850?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/362643917935?nordt=true&orig_cvip=true&rt=nc',
 nan,
 'https://www.ebay.com/itm/362643928499?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/254343826760?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/383240884447?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/40327

In [4]:
while np.nan in Links:
    Links.remove(np.nan)
None in Links

False

In [5]:
len(Links)

1755

In [ ]:
# настраиваем драйвер
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen

# Настраиваем браузер, чтобы он не грузил Java и картинки
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options, chrome_options=chrome_options)

# Делаем запрос
driver.get("https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=Smartphones&dayRange=730&endDate=1684051559220&startDate=1620979559220&categoryId=9355&offset=0&limit=50&tabName=SOLD&tz=Europe%2FMoscow")

#Атдыхаем, ждём загрузку
time.sleep(5)

In [16]:
characteristics_df = pd.DataFrame()

In [ ]:
unavaible = []
for cur_link in tqdm(Links[0:1200]):
        if cur_link!= None:
            try:
                driver.get(cur_link)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(randint(1, 5))
                html = driver.page_source
                soup2 = BeautifulSoup(html, 'html.parser')
                title_tag = soup2.find('h1', {'class': 'x-item-title__mainTitle'})
                title_text = title_tag.text.strip()
                parent_element = driver.find_element(By.CLASS_NAME, 'ux-layout-section--features')
                characteristics = parent_element.text.split("\n")
                characteristics_data = {"Title": title_text, "Link": cur_link}
                for i in range(len(characteristics)):
                    if i%2==0:
                        name = characteristics[i]
                    else:
                        value = characteristics[i]
                        characteristics_data[name] = value

                characteristics_df = characteristics_df.append(characteristics_data, ignore_index=True)
            except:
                print("Сорян, не полусилось:", cur_link)
                unavaible.append(cur_link)
        pass

characteristics_df.to_csv("characteristics_df.tsv", sep='\t', index=False)

  0%|          | 0/1200 [00:00<?, ?it/s]

In [20]:
characteristics_df

,Title,Link,Condition,Seller Notes,about the seller notes,Apple,iPhone 11,64GB,Factory Unlocked,12MP,...,12.0MP,iPhone 7 Plus,iOS - Apple,"4G, Bluetooth, GPS, Wi-Fi","Internet Browser, 4G Data Capable, Touchscreen, 4K Video Recording, Bluetooth Enabled, GPS, Music Player, Speakerphone",A1784,"MN572LL/A, MN522LL/A, MN562LL/A, MN532LL/A",4G SIM Card Required - NOT included,"Gold, Rose Gold, Silver, Black, Jet Black, Red",Quad Core
0,Apple iPhone 11 64GB Factory Unlocked 4G LTE S...,https://www.ebay.com/itm/254604777645?nordt=tr...,Very Good - Refurbished,“This iPhone 11 is in Very Good condition and ...,Brand,Model,Storage Capacity,Network,Camera Resolution,Screen Size,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone XR 64GB Factory Unlocked Smartpho...,https://www.ebay.com/itm/254187678666?nordt=tr...,Very Good - Refurbished,“This Apple iPhone XR 64GB Factory Unlocked Sm...,Brand,Model,NaN,Network,Camera Resolution,Screen Size,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone 11 64GB Unlocked Smartphone - Ver...,https://www.ebay.com/itm/363183815277?nordt=tr...,Very Good - Refurbished,"“This is a B+ Stock item, meaning unit is in v...",Camera Resolution,Memory Card Type,NaN,NaN,MPN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone X 64GB Factory Unlocked Phone - V...,https://www.ebay.com/itm/382605209867?nordt=tr...,Very Good - Refurbished,"“This is a B+ Stock item, meaning unit is in v...",Camera Resolution,Memory Card Type,NaN,SIM Card Slot,Network Technology,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone X 256GB Unlocked Smartphone - Ver...,https://www.ebay.com/itm/382446518910?nordt=tr...,Very Good - Refurbished,"“This is a B+ Stock item, meaning unit is in v...",Camera Resolution,Memory Card Type,NaN,NaN,Network Technology,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Samsung G960 Galaxy S9 64GB Factory Unlocked S...,https://www.ebay.com/itm/401763851775?nordt=tr...,Very Good - Refurbished,"“This is a B+ Stock item, meaning unit is in v...",Camera Resolution,NaN,NaN,SIM Card Slot,Manufacturer Color,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Apple iPhone 8 Plus 64GB Unlocked Smartphone -...,https://www.ebay.com/itm/362534145919?nordt=tr...,Very Good - Refurbished,"“This is a B+ Stock item, meaning unit is in v...",Camera Resolution,Memory Card Type,NaN,SIM Card Slot,Network Technology,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Apple iPhone 11 64GB Factory Unlocked 4G LTE S...,https://www.ebay.com/itm/254733458977?nordt=tr...,Good - Refurbished,“This iPhone 11 is 100% functional in Good con...,Brand,Model,Storage Capacity,Network,Camera Resolution,Screen Size,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Samsung Galaxy S10 G973U 128GB Factory Unlocke...,https://www.ebay.com/itm/254449547850?nordt=tr...,Very Good - Refurbished,“This Galaxy S10 is in Very Good condition and...,Brand,NaN,NaN,NaN,Camera Resolution,Screen Size,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Samsung G965 Galaxy S9+ Plus 64GB Factory Unlo...,https://www.ebay.com/itm/362643917935?nordt=tr...,Very Good - Refurbished,"“This is a B+ Stock item, meaning unit is in v...",Camera Resolution,NaN,NaN,SIM Card Slot,Manufacturer Color,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
characteristics_df.to_csv("characteristics_df.tsv", sep='\t', index=False)

Пытаемся сделать многопоточность, но, кажется, не получилось(

In [ ]:
import concurrent.futures

def parser(x):
    characteristics_df = pd.DataFrame()

    # настраиваем драйвер
    options = Options()
    options.headless = True  # hide GUI
    options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
    options.add_argument("start-maximized")  # ensure window is full-screen

    # Настраиваем браузер, чтобы он не грузил Java и картинки
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options, chrome_options=chrome_options)

    time.sleep(5)

    unavaible = []
    start = int((1200*x)/3)
    stop = int(1200*(1+x)/3)

    for cur_link in tqdm(Links[start:stop]):
        if cur_link!= None:
            try:
                driver.get(cur_link)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(randint(1, 5))
                html = driver.page_source
                soup2 = BeautifulSoup(html, 'html.parser')
                title_tag = soup2.find('h1', {'class': 'x-item-title__mainTitle'})
                title_text = title_tag.text.strip()
                parent_element = driver.find_element(By.CLASS_NAME, 'ux-layout-section--features')
                characteristics = parent_element.text.split("\n")
                characteristics_data = {"Title": title_text, "Link": cur_link}
                for i in range(len(characteristics)):
                    if i%2==0:
                        name = characteristics[i]
                    else:
                        value = characteristics[i]
                        characteristics_data[name] = value

                characteristics_df = characteristics_df.append(characteristics_data, ignore_index=True)
            except:
                print("Сорян, не полусилось:", cur_link)
                unavaible.append(cur_link)
        pass
    driver.quit()
    characteristics_df.to_csv(f"characteristics_df {x}.tsv", sep='\t', index=False)

# создаем пул потоков
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    # запускаем функцию run_selenium() в каждом потоке
    futures = [executor.submit(parser(i)) for i in range(3)]
    # дожидаемся завершения всех потоков
    concurrent.futures.wait(futures)